daily_run estimates strikes, number of contracts for a put selling strategy on S&P 500 based on "Cost of Capital for Alternatives" - Data requires either an open session of IB Gateway or TWS from Inter-active brokers

In [1]:
from option_daily_prod import OptionMarket, SpxOptionAsset, RSL2OptionAsset
import numpy as np
import pandas as pd
import nest_asyncio
from time import time

before = time()
nest_asyncio.apply()

spx_trades = OptionMarket(SpxOptionAsset())
rsl_trades = OptionMarket(RSL2OptionAsset())

In [5]:
z_score = np.array([-1, -1.5, -2])
leverage = np.array(range(1, 4, 1))
num_expiries = 3
illiquid_equity = 500000
after = time()
spx = True
if spx:
    trd_choice = spx_trades.form_trade_choice(z_score, num_expiries, 'P')
else:
    trd_choice = rsl_trades.form_trade_choice(z_score, num_expiries, 'P')
    
last_price = trd_choice.spot
after_2 = time()
print('elapsed:', np.round(after - before, 1))
print('elapsed:', np.round(after_2 - before, 1))
print(last_price)
print(trd_choice.sigma)
print(trd_choice.account_value)

elapsed: 51.9
elapsed: 64.0
2670.72
19.21
[AccountValue(account='U2463043', tag='NetLiquidationByCurrency', value='2774291.3424', currency='BASE', modelCode='')]


In [6]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction:row;
}
"""
HTML('<style>{}</style>'.format(CSS))


In [7]:
_=[display(df) for df in [trd_choice.strike_grid(), trd_choice.premium_grid(), trd_choice.prices_grid()]]

,-1.0,-1.5,-2.0
2018.12.21,2540,2480,2440
2019.01.18,2475,2395,2330
2019.02.15,2415,2305,2230


,-1.0,-1.5,-2.0
2018.12.21,14.30,8.00,5.60
2019.01.18,20.75,12.75,8.65
2019.02.15,25.95,15.40,10.80


,-1.0,-1.5,-2.0
2018.12.21,14.2/14.4,7.9/8.1,5.5/5.7
2019.01.18,20.5/21.0,12.6/12.9,8.5/8.8
2019.02.15,25.6/26.3,15.2/15.6,10.6/11.0


In [8]:

prct_decline = trd_choice.pct_otm_grid(last_price)
format_dict = {x: '{:.2%}' for x in prct_decline.to_dict()}
display(prct_decline.style.format(format_dict))

,-1.0,-1.5,-2.0
2018.12.21,-4.89%,-7.14%,-8.64%
2019.01.18,-7.33%,-10.32%,-12.76%
2019.02.15,-9.57%,-13.69%,-16.50%


In [9]:
capital_at_risk = float(trd_choice.account_value[0].value) + illiquid_equity
premium_list = []
premium_list =  [(trd_choice.premium_grid() * contractTable.round() * 100) \
                 for contractTable in trd_choice.option_lots(leverage, capital_at_risk)]

_ = [display(df.style.format('${:,.0f}')) for df in premium_list]  

,-1.0,-1.5,-2.0
2018.12.21,"$18,590","$10,400","$7,280"
2019.01.18,"$26,975","$17,850","$12,110"
2019.02.15,"$36,330","$21,560","$16,200"


,-1.0,-1.5,-2.0
2018.12.21,"$37,180","$21,600","$15,120"
2019.01.18,"$56,025","$35,700","$24,220"
2019.02.15,"$72,660","$44,660","$32,400"


,-1.0,-1.5,-2.0
2018.12.21,"$55,770","$32,000","$22,400"
2019.01.18,"$83,000","$52,275","$36,330"
2019.02.15,"$106,395","$66,220","$47,520"


In [10]:
premium_list_pct = []
premium_list_pct = [(trd_choice.premium_grid() * lev) / last_price for lev in leverage]
_ = [display(df.style.format('{:.2%}')) for df in premium_list_pct]  

,-1.0,-1.5,-2.0
2018.12.21,0.54%,0.30%,0.21%
2019.01.18,0.78%,0.48%,0.32%
2019.02.15,0.97%,0.58%,0.40%


,-1.0,-1.5,-2.0
2018.12.21,1.07%,0.60%,0.42%
2019.01.18,1.55%,0.95%,0.65%
2019.02.15,1.94%,1.15%,0.81%


,-1.0,-1.5,-2.0
2018.12.21,1.61%,0.90%,0.63%
2019.01.18,2.33%,1.43%,0.97%
2019.02.15,2.91%,1.73%,1.21%


In [11]:

_ = [display(contractTable.round().style.format('{:.0f}')) for \
     contractTable in trd_choice.option_lots(leverage, capital_at_risk)]
    

,-1.0,-1.5,-2.0
Lev 1,,,
2018.12.21,13,13,13
2019.01.18,13,14,14
2019.02.15,14,14,15


,-1.0,-1.5,-2.0
Lev 2,,,
2018.12.21,26,27,27
2019.01.18,27,28,28
2019.02.15,28,29,30


,-1.0,-1.5,-2.0
Lev 3,,,
2018.12.21,39,40,40
2019.01.18,40,41,42
2019.02.15,41,43,44


In [12]:
margin_dlr = [trd_choice.margin(last_price) * df for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in margin_dlr]  

,-1.0,-1.5,-2.0
2018.12.21,"$543,041","$456,851","$401,731"
2019.01.18,"$466,926","$379,644","$338,310"
2019.02.15,"$426,124","$344,260","$350,700"


,-1.0,-1.5,-2.0
2018.12.21,"$1,086,082","$948,845","$834,365"
2019.01.18,"$969,770","$759,287","$676,620"
2019.02.15,"$852,247","$713,110","$701,400"


,-1.0,-1.5,-2.0
2018.12.21,"$1,629,124","$1,405,696","$1,236,096"
2019.01.18,"$1,436,696","$1,111,813","$1,014,930"
2019.02.15,"$1,247,933","$1,057,370","$1,028,720"


In [22]:
marge = trd_choice.margin(trd_choice.spot)
lots = trd_choice.option_lots([2], capital_at_risk)
margin = lots[0] * marge
display(margin.style.format('${:,.0f}'))

,-1.0,-1.5,-2.0
Lev 2,,,
2018.12.21,"$1,086,082","$948,845","$834,365"
2019.01.18,"$969,770","$759,287","$676,620"
2019.02.15,"$852,247","$713,110","$701,400"


In [14]:
notional_dlr=[df * trd_choice.strike_grid() * 100 for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in notional_dlr]  

,-1.0,-1.5,-2.0
Lev 1,,,
2018.12.21,"$3,302,000","$3,224,000","$3,172,000"
2019.01.18,"$3,217,500","$3,353,000","$3,262,000"
2019.02.15,"$3,381,000","$3,227,000","$3,345,000"


,-1.0,-1.5,-2.0
Lev 2,,,
2018.12.21,"$6,604,000","$6,696,000","$6,588,000"
2019.01.18,"$6,682,500","$6,706,000","$6,524,000"
2019.02.15,"$6,762,000","$6,684,500","$6,690,000"


,-1.0,-1.5,-2.0
Lev 3,,,
2018.12.21,"$9,906,000","$9,920,000","$9,760,000"
2019.01.18,"$9,900,000","$9,819,500","$9,786,000"
2019.02.15,"$9,901,500","$9,911,500","$9,812,000"
